In [ ]:
from scipy.io.wavfile import read
SAMPLING_RATE, audio = read("mhll.wav")
BPM = 60

In [8]:
def getWindow():
    # Convert BPM to approximate number of samples per beat
    interval_seconds = (60 / BPM) / 8 
    #32nd note length
    sample_count = interval_seconds * SAMPLING_RATE
    window = {}
    window["sample_count"] = sample_count
    window["interval_seconds"]= interval_seconds
    return window

def splitAudio():
    split_audio = {}
    overlap = 0.5
    start_sample = 0

    # Assuming `w["sample_count"]` and `audio` are properly defined
    sample_count = int(w["sample_count"])

    while start_sample + sample_count <= len(audio): 
        # Slice the segment from the audio
        segment = audio[start_sample : start_sample + sample_count]

        # Create dictionary with time as key
        split_audio[start_sample / SAMPLING_RATE] = segment

        # Move start_sample by overlap
        start_sample = start_sample + int(sample_count * (1 - overlap))
        
        if start_sample + sample_count >= len(audio): 
            segment = audio[start_sample: len(audio)]
            split_audio[start_sample / SAMPLING_RATE] = segment

    return split_audio


In [ ]:
w = getWindow()
split_audio = splitAudio()

##FFT ALFORITHm

In [1]:
import cmath

In [4]:
def DFT(window): 
    numSamp = len(window)
    DFT_result = {}
    # Loop through frequencies up to Nyquist
    for freq in range(numSamp):
        # Initialize the coefficient for the frequency
        coefficient = 0
        for sample in range(numSamp): 
            # Use cmath.exp for complex exponential
            coefficient += window[sample] * cmath.exp(-2j * cmath.pi * freq / numSamp * sample)
        DFT_result[freq] = coefficient
    return DFT_result

In [ ]:
           
def findDistance(note1, note2): 
    distance = 0
    #*not accounting for # or flats
    n1_letter_val  = ord(note1[0])
    n1_octave  = int(note1[1])
    n2_letter_val  = ord(note2[0])
    n2_octave  = int(note2[1])
    
    distance = np.abs(n1_letter_val - n2_letter_val) + 8 * np.abs(n1_octave - n2_octave)
    
    
    # print("Letter B: ", n1_letter_val, " Octave: ", n1_octave)
    # print("Letter C: ", n2_letter_val, " Octave: ", n2_octave)
    # print("Distance: ", distance)
    return distance

def findClef(score): 
    clefs = {
        "treble": "B5",
        "bass": "D3",
        "alto": "C4"
    }
    measure = 0
    min = 9999999999999
    closestClef = ""
    for clef in clefs: 
        middlenote = clefs[clef]
        for data in score:
            distance = findDistance(data[1], middlenote)
            #* emphasize close notes more
            measure += math.exp(distance)
        
        if measure <= min: 
            min = measure
            closestClef = clef
    return closestClef


def findKeySignature(): 
    keySignatures = { 
        # Key Signatures in the Treble Clef
        "C Major": ["C", "D", "E", "F", "G", "A", "B"],  # No sharps or flats
        "G Major": ["G", "A", "B", "C", "D", "E", "F#"],  # One sharp (F#)
        "D Major": ["D", "E", "F#", "G", "A", "B", "C#"],  # Two sharps (F#, C#)
        "A Major": ["A", "B", "C#", "D", "E", "F#", "G#"],  # Three sharps (F#, C#, G#)
        "E Major": ["E", "F#", "G#", "A", "B", "C#", "D#"],  # Four sharps (F#, C#, G#, D#)
        "B Major": ["B", "C#", "D#", "E", "F#", "G#", "A#"],  # Five sharps (F#, C#, G#, D#, A#)
        "F# Major": ["F#", "G#", "A#", "B", "C#", "D#", "E#"],  # Six sharps (F#, C#, G#, D#, A#, E#)
        "C# Major": ["C#", "D#", "E#", "F#", "G#", "A#", "B#"],  # Seven sharps (F#, C#, G#, D#, A#, E#, B#)
        "F Major": ["F", "G", "A", "Bb", "C", "D", "E"],  # One flat (Bb)
        "Bb Major": ["Bb", "C", "D", "Eb", "F", "G", "A"],  # Two flats (Bb, Eb)
        "Eb Major": ["Eb", "F", "G", "Ab", "Bb", "C", "D"],  # Three flats (Bb, Eb, Ab)
        "Ab Major": ["Ab", "Bb", "C", "Db", "Eb", "F", "G"],  # Four flats (Bb, Eb, Ab, Db)
        "Db Major": ["Db", "Eb", "F", "Gb", "Ab", "Bb", "C"],  # Five flats (Bb, Eb, Ab, Db, Gb)
        "Gb Major": ["Gb", "Ab", "Bb", "Cb", "Db", "Eb", "F"],  # Six flats (Bb, Eb, Ab, Db, Gb, Cb)
        "Cb Major": ["Cb", "Db", "Eb", "Fb", "Gb", "Ab", "Bb"],  # Seven flats (Bb, Eb, Ab, Db, Gb, Cb, Fb
                     }
    
    for key_sig in keySignatures:
        count = 0
        min = 999
        bestKey = ""
        for data in score: 
            note = data[1]
            note_name = note[0]
            #print(note_name)
            if note_name not in keySignatures[key_sig]: 
                count += 1 
        if count <= min: 
            bestKey = key_sig
        
    return bestKey
                                                                                                         
print("Closest Clef: ", findClef(score))
print("Key Signature: ", findKeySignature())

In [7]:
LEVEL =0 
twiddles = {}
signal = [0,1,2,3,4,5,6,7]
N = len(signal)
#intialize 
for sample in signal: 
    twiddles[sample] = 1
    
def getTwiddles(level, *args): 
    if len(even ) == 1: 
            return
    for list in args: 
        even = list[::2]
        odd = list[1::2]
        level += 1
        for sample in odd: 
            twiddles[sample] *= cmath.exp((-1j * cmath.pi  * 2 * level)/N)
        
        
        getTwiddles(level, even, odd)
        
def computeFFT(): 
    frequencyBins = {}
    for frequency in range(N): 
        sum = 0
        frequencyBins[frequency] = 0
        for sample in range(N): 
            frequencyBins[frequency] += signal[sample] * twiddles[sample]**frequency
    
    return frequencyBins
        
getTwiddles(LEVEL,signal)   
print(twiddles)

for freq in computeFFT(): 
    print("Frequency bin: ", freq, "value: ", computeFFT()[freq])
    
print(DFT(signal))
    

{0: 1, 1: (0.7071067811865476-0.7071067811865475j), 2: (6.123233995736766e-17-1j), 3: (-1-1.7938038903913487e-16j), 4: (-0.7071067811865475-0.7071067811865476j), 5: (-0.7071067811865477+0.7071067811865474j), 6: (6.123233995736766e-17-1j), 7: (-1-1.7938038903913487e-16j)}
Frequency bin:  0 value:  (28+0j)
Frequency bin:  1 value:  (-15.65685424949238-8.000000000000002j)
Frequency bin:  2 value:  (2.0000000000000018-1.9999999999999973j)
Frequency bin:  3 value:  (-4.343145750507623+7.999999999999998j)
Frequency bin:  4 value:  (8+2.704382594388419e-15j)
Frequency bin:  5 value:  (-4.343145750507615-8.000000000000004j)
Frequency bin:  6 value:  (1.9999999999999956+2.000000000000008j)
Frequency bin:  7 value:  (-15.656854249492378+7.9999999999999805j)
{0: (28+0j), 1: (-4.0000000000000036+9.65685424949238j), 2: (-4.000000000000002+3.9999999999999987j), 3: (-4.000000000000001+1.656854249492385j), 4: (-4-3.4290110376125885e-15j), 5: (-4.000000000000009-1.6568542494923904j), 6: (-4.00000000000